<a href="https://colab.research.google.com/github/dubisx/EMCT_final/blob/main/tf_transformer_model_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers
!pip install miditok
!pip install tqdm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 58.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 69.1 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.4/94.4 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.1/51.1 kB 7.7 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import tensorflow as tf
import transformers
import miditok
import numpy
import os
import pathlib
from tensorflow import keras
import json
from transformers import TFGPT2LMHeadModel, GPT2Config, Trainer, TrainingArguments, GenerationConfig
from miditok import MIDILike, MIDITokenizer, REMI
from miditok.constants import CHORD_MAPS
from miditoolkit import MidiFile
import tqdm

In [3]:
attn_window = 10
j = 0
input_seq = [] 
target_seq = []
batch_size = 16
buffer_size = 1000


In [4]:
from google.colab import drive

drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [6]:
directory = "/content/gdrive/MyDrive/ala/REMI/tokens_BPE/"

for fname in os.listdir(directory):                     # for each file in the directory
    with open(os.path.join(directory, fname)) as i:     # open the file
        data = json.load(i)                             # load the json file                             

    for i in range(0, len(data['ids'][0])-1, 11):    # for each token in the file
            x = data['ids'][0][i:i+attn_window]      # get the next 10 tokens
            y = data['ids'][0][i+1:i+attn_window+1]  # get the next 1+10 tokens

            if len(x) == attn_window and len(y) == attn_window: # if the length of the tokens is 10
                
                print('batch number:', j)            # print the batch number
                print('input:', x)                   
                print('target:', y) 
                print('-----')

                j += 1                               # increment the batch number

                input_seq.append(x)                  # append the input sequence
                target_seq.append(y)                 # append the target sequence

Streaming output truncated to the last 5000 lines.
batch number: 469020
input: [127, 347, 11, 92, 131, 350, 8, 94, 127, 343]
target: [347, 11, 92, 131, 350, 8, 94, 127, 343, 15]
-----
batch number: 469021
input: [92, 139, 360, 8, 94, 127, 342, 13, 92, 131]
target: [139, 360, 8, 94, 127, 342, 13, 92, 131, 352]
-----
batch number: 469022
input: [8, 94, 127, 339, 8, 92, 131, 27, 698, 367]
target: [94, 127, 339, 8, 92, 131, 27, 698, 367, 10]
-----
batch number: 469023
input: [94, 127, 347, 11, 92, 131, 350, 8, 94, 127]
target: [127, 347, 11, 92, 131, 350, 8, 94, 127, 343]
-----
batch number: 469024
input: [15, 92, 131, 360, 8, 94, 127, 342, 16, 92]
target: [92, 131, 360, 8, 94, 127, 342, 16, 92, 131]
-----
batch number: 469025
input: [356, 33, 524, 38, 115, 167, 369, 57, 824, 69]
target: [33, 524, 38, 115, 167, 369, 57, 824, 69, 824]
-----
batch number: 469026
input: [365, 57, 567, 69, 567, 359, 57, 483, 69, 376]
target: [57, 567, 69, 567, 359, 57, 483, 69, 376, 397]
-----
batch number: 46

In [7]:
print('input_seq length:', len(input_seq))
print('target_seq length:', len(target_seq))

input_seq length: 470270
target_seq length: 470270


In [8]:
from sklearn.model_selection import train_test_split

train_size = 0.8 # 80/20 split between train and test+val data
test_size = 0.5 # 50/50 (80/10/10) split between train, test and val

x_train, x_rem, y_train, y_rem = train_test_split(input_seq, target_seq, train_size = 0.8)
x_test, x_val, y_test, y_val = train_test_split(x_rem, y_rem, test_size = 0.5)


In [9]:
print('train input:', len(x_train)), print('train target:', len(y_train))
print('val input:', len(x_val)), print('val target:', len(y_val))
print('test input:', len(x_test)), print('test target:', len(y_test))

train input: 376216
train target: 376216
val input: 47027
val target: 47027
test input: 47027
test target: 47027


(None, None)

In [19]:
#creates tokenizer
tokenizer = REMI(params = "/content/gdrive/MyDrive/ala/REMI/params.json")

In [20]:
# Creates model
config = GPT2Config( 
    vocab_size=1000,
    n_positions=attn_window,
    n_embd=64,
    n_layer=4,
    n_head=4,
    n_inner=128,
    resid_pdrop=.1,
    embd_pdrop=.1,
    attn_pdrop=.1,
    pad_token_id=tokenizer['PAD_None'],
    bos_token_id=tokenizer['BOS_None'],
    eos_token_id=tokenizer['EOS_None'],
)
model = TFGPT2LMHeadModel(config)                      

In [ ]:
#model.from_pretrained("/content/gdrive/MyDrive/model_f/model2/")

In [21]:
# defining our optimizer
optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08, clipnorm=1.0)
# definining our loss function
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
# defining our metric which we want to observe
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
# compiling the model
model.compile(optimizer=optimizer, loss=[loss, *[None] * model.config.n_layer], metrics=[metric])


In [22]:
num_epoch = 150

# Create a callback that saves the model's weights
cp_callback = [

# tf.keras.callbacks.ModelCheckpoint("/content/gdrive/MyDrive/models_1/",
#                                                  verbose=1)

tf.keras.callbacks.LambdaCallback(on_epoch_end=lambda epoch, logs: model.save_pretrained("/content/gdrive/MyDrive/model_f/checkpoints/REMI_FD_model_1"))
]

history = model.fit(x_train, y_train, epochs=num_epoch, callbacks=[cp_callback], validation_data = (x_val, y_val))


Epoch 1/150
11757/11757 [==============================] - 288s 23ms/step - loss: 5.3427 - accuracy: 0.0473 - val_loss: 4.5037 - val_accuracy: 0.0945
Epoch 2/150
11757/11757 [==============================] - 259s 22ms/step - loss: 4.2471 - accuracy: 0.1255 - val_loss: 3.9131 - val_accuracy: 0.1709
Epoch 3/150
11757/11757 [==============================] - 256s 22ms/step - loss: 3.8437 - accuracy: 0.1886 - val_loss: 3.6081 - val_accuracy: 0.2309
Epoch 4/150
11757/11757 [==============================] - 257s 22ms/step - loss: 3.6256 - accuracy: 0.2287 - val_loss: 3.4445 - val_accuracy: 0.2571
Epoch 5/150
11757/11757 [==============================] - 255s 22ms/step - loss: 3.5052 - accuracy: 0.2481 - val_loss: 3.3493 - val_accuracy: 0.2705
Epoch 6/150
11757/11757 [==============================] - 261s 22ms/step - loss: 3.4282 - accuracy: 0.2599 - val_loss: 3.2819 - val_accuracy: 0.2801
Epoch 7/150
11757/11757 [==============================] - 262s 22ms/step - loss: 3.3720 - accuracy:

In [33]:
model.save_pretrained('/content/gdrive/MyDrive/model_f/REMI_FD_model_2', saved_model = True)
model.save_pretrained('/content/gdrive/MyDrive/model_f/REMI_FD_model_1')
model.save_weights('/content/gdrive/MyDrive/model_f/REMI_weights')

In [37]:
model.evaluate(x_test, y_test)

1470/1470 [==============================] - 9s 6ms/step - loss: 2.8818 - accuracy: 0.3406


[2.881814479827881, 0.3406064510345459]

In [38]:
model1 = TFGPT2LMHeadModel(config)     
model1.from_pretrained('/content/gdrive/MyDrive/model_f/REMI_FD_model_2')
model1.load_weights('/content/gdrive/MyDrive/model_f/REMI_weights')    

model1.compile(optimizer=optimizer, loss=[loss, *[None] * model.config.n_layer], metrics=[metric])
model1.evaluate(x_test, y_test)

All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at /content/gdrive/MyDrive/model_f/REMI_FD_model_2.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


1470/1470 [==============================] - 10s 5ms/step - loss: 2.8818 - accuracy: 0.3406


[2.881814479827881, 0.3406064510345459]